<h1 align="center">Why is Python so <i>slow</i>?</h1>
<h2 align="center">(and what can I do about it?)</h2>
&nbsp;
<div align="center">
  <img src="https://ichef.bbci.co.uk/images/ic/640x360/p06vd19l.jpg" />
</div>

<h2 align="center">I don't care. I only use FORTRAN (or C++)</h2>

<div align="center">
  <img src="https://www.fourmilab.ch/documents/univac/figures/card-90col.gif" />
</div>

<h2 align="center">But Python isn't slow!</h2>

<div align="center">
  <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/c/cf/Casio_calculator_JS-20WK_in_201901_002.jpg/533px-Casio_calculator_JS-20WK_in_201901_002.jpg" />
</div>

<h2 align="center">I <i>like</i> that Python is slow!</h2>

<div align="center">
  <img width="600px" src="figures/1_PAY-Boris-Johnson-and-staff-pictured-with-wine-in-Downing-Street-garden-in-May-2020.jpg" />
</div>

## So why _is_ Python slow?
### Python is _*interpreted*_.

<table align="center">
    <tr>
        <td>
  <img width="500px" src="https://csharpcorner-mindcrackerinc.netdna-ssl.com/article/why-learn-python-an-introduction-to-python/Images/last2.png" float="left"/></td>
        <td><img width="500px" src="figures/Compiled.jpg" float="right"/></td>
    </tr>
    </table>


## So why _is_ Python slow?
### Python is _*dynamically typed*_.

In [ ]:
import numpy as np

# Add a random number of objects with random types to the list
def mess_with_my_list(list_of_vars):
    types = [int, float, str]
    values = np.random.uniform(1, 10, np.random.randint(0, 5))
    selector = np.random.randint(0, 3, len(values))
    for selected, value in zip(selector, values):
        list_of_vars.append(types[selected](value))

<div align="center">
  <img width="600px" src="https://images.theconversation.com/files/298413/original/file-20191023-119419-x4f7cv.JPG" />
</div>

In [ ]:
list_of_vars = []

for i in range(3):
    mess_with_my_list(list_of_vars)
    print(i, "=>", list_of_vars)

## How slow _is_ slow?

* Time for some magic! 

In [ ]:
%%timeit -r 1 -n 4

a = np.random.uniform(0, 10)
b = np.random.uniform(0, 10)
c = np.random.uniform(0, 10)

positive_soln = -b + np.lib.scimath.sqrt(b**2 - 4*a*c)/ 2*a
negative_soln = -b - np.lib.scimath.sqrt(b**2 - 4*a*c)/ 2*a

print(f"x = {[positive_soln, negative_soln]}")

## How slow _is_ slow?

* A "quick" example - adding up some numbers.

In [ ]:
numbers = np.random.uniform(0, 1, (500, 500, 500))
print(numbers.shape)
print(numbers.size)

In [ ]:
%%timeit -r 3 -n 1
total = 0
for i in range(500):
    for j in range(500):
        for k in range(500):
            total += numbers[i, j, k]
print(total)

<div align="center">
  <img width="400px" src="https://upload.wikimedia.org/wikipedia/en/5/52/Testcard_F.jpg" />
</div>


## How can we do better?

* Compile your Python?

In [ ]:
import numba

In [ ]:
@numba.jit(nopython=True)
def add_em_up(numbers):
    total = 0
    for i in range(500):
        for j in range(500):
            for k in range(500):
                total += numbers[i,j,k]
    return total

In [ ]:
%%timeit -r 10 -n 1
total = add_em_up(numbers)
print(total)                

<div align="center">
  <img width="400px" src="https://c.tenor.com/41I-iMyClCgAAAAd/programmer-programming.gif" />
</div>

## How can we do better?

* Use Python builtin functions e.g. `sum`.

In [ ]:
%%timeit -r 10 -n 1

# Need to call sum three times - once for each dimension
total = sum(sum(sum(numbers)))
            
print(total)

## How can we do better?

* Use `numpy` functions (or `scipy`, `pandas` etc).

In [ ]:
%%timeit -r 10 -n 1

total = numbers.sum()
            
print(total)

<div align="center">
  <img width="400px" src="https://media4.giphy.com/media/4xpB3eE00FfBm/giphy.gif" />
</div>

# Effective `numpy`

<div align="center">
  <img width="400px" src="https://media4.giphy.com/media/unQ3IJU2RG7DO/giphy.gif" />
</div>

## Brief aside...

* If you want to **build** a `list` or `dict` of values comprehension expressions can **sometimes** be faster than for loops

In [ ]:
flat_numbers = numbers.flatten()

### List comprehension
Using a **list comprehension** expression, we can write a for loop that builds a `list`...

In [ ]:
%%timeit -r 1 -n 1
squares = []
for i in range(len(flat_numbers)):
    squares.append(flat_numbers[i] * flat_numbers[i])
    
print(*squares[:10], sep="\n")

... like this

In [ ]:
%%timeit -r 1 -n 1
squares = [flat_numbers[i] * flat_numbers[i] for i in range(len(flat_numbers))]

### Dict comprehension
Using a **dict comprehension** expression, we can write a for loop that builds a `dict`...

In [ ]:
%%timeit -r 1 -n 1
squares = dict()
for i in range(len(flat_numbers)//3):
    squares[i] = flat_numbers[i] * flat_numbers[i]
    
print(*[squares[i] for i in range(10)], sep="\n")

... like this

In [ ]:
%%timeit -r 1 -n 1
squares = {i : flat_numbers[i] * flat_numbers[i] for i in range(len(flat_numbers)//3)}

## Vectorised operations

In [ ]:
a = np.random.uniform(size=flat_numbers.size)
b = np.random.uniform(size=flat_numbers.size)
c = np.random.uniform(size=flat_numbers.size)

### The old-fashioned way
Processing **100th** of the total data (I only have 40 mins!)

In [ ]:
%%timeit -r 1 -n 1
y = []
for i in range(a.size//100):
    y.append(-b[i] + np.lib.scimath.sqrt(b[i]**2 - 4*a[i]*c[i])/ 2*a[i])
    
print(y[:20])

<div align="center">
  <img width="400px" src="https://c.tenor.com/Z4t8sZCrOuMAAAAM/riker.gif" />
</div>

### Now with `numpy` vectorised ops.
We'll do the lot this time!

In [ ]:
%%timeit -r 1 -n 1

y = -b + np.lib.scimath.sqrt(b**2 - 4*a*c)/ 2*a

print(list(y[:20]))

<div align="center">
  <img width="400px" src="https://c.tenor.com/DBqjevyA2o4AAAAd/bongo-cat-codes.gif" />
</div>

## The dreaded stack trace

In [ ]:
a = np.random.uniform(size=(4,2,3))
b = np.random.uniform(size=(4,3))
print(a, b, sep="\n\n")

In [ ]:
c = a * b

In [ ]:
result = []

for i in range(2):
    result.append(a[:,i,:]*b)

print(np.array(result))

<div align="center">
  <img width="400px" src="https://c.tenor.com/LRtRSo1uhQsAAAAC/star-trek-picard.gif" />
</div>

## The `shape` of numpy arrays

<div align="center">
  <img width="700px" src="https://i.stack.imgur.com/NWTQH.png" />
</div>

In [ ]:
print(a.shape)
print(a.sum(axis=0).shape)
print(a.prod(axis=1).shape)
print(a.cumsum(axis=2).shape)

`numpy` mathematical function reference [https://numpy.org/doc/stable/reference/routines.math.html](https://numpy.org/doc/stable/reference/routines.math.html)

## The `shape` of numpy arrays

An array with shape `(4,3,2)` is a _nested_ data structure - an array of 4 arrays of 3 arrays of 2 elements.

<div align="center">
  <img width="400px" src="https://c.tenor.com/bvfEFc8xI_0AAAAd/llama-stare.gif" />
</div>

## The `shape` of numpy arrays

An array of 4 arrays of 3 arrays of 2 elements.

In [ ]:
np.random.random(size=(4,3,2))

## Broadcasting
* How `numpy` performs arithmetic on arrays with different shapes.
* The **innermost** nested dimensions must match.

In [ ]:
print(a.shape)

In [ ]:
a* np.random.uniform(size=3)

In [ ]:
a * np.random.uniform(size=(2, 3))

In [ ]:
a * np.random.uniform(size=(4, 2, 3))

## Coercing shapes with `rollaxis` and `swapaxes`
&nbsp;
<div align="center">
  <img width="400px" src="https://c.tenor.com/3rGvpHXONYYAAAAC/tesseract.gif" />
</div>


In [ ]:
print("a", a.shape, "b", b.shape, sep="\n")

`rollaxis` moves the selected dimension to the "outside".

In [ ]:
rolled = np.rollaxis(a, 1)
print(rolled.shape, np.array(result).shape)
rolled * b == np.array(result)

`swapaxes` swaps two axes - surprise, surprise!

In [ ]:
swapped = np.swapaxes(a, 0, 1)
print(swapped.shape, np.array(result).shape)
swapped * b == np.array(result)

Other examples like `transpose` (`T`), `moveaxis` at [https://numpy.org/doc/stable/reference/routines.array-manipulation.html#transpose-like-operations](https://numpy.org/doc/stable/reference/routines.array-manipulation.html#transpose-like-operations)

## Applying functions

&nbsp;
<div align="center">
  <img width="400px" src="https://c.tenor.com/AnWNRWE78A8AAAAM/apply-apply-apply-apply.gif" />
</div>

Efficiently computing values using subsets of a multi-dimensional array.

In [ ]:
from astropy.cosmology import Planck18_arXiv_v2

In [ ]:
def angular_diameter_distance_range(redshifts):
    min_dist = Planck18_arXiv_v2.angular_diameter_distance(redshifts.min())
    max_dist = Planck18_arXiv_v2.angular_diameter_distance(redshifts.max())
    return max_dist - min_dist

In [ ]:
a

In [ ]:
np.apply_along_axis(angular_diameter_distance_range, 2, a)

Be **careful** with applying your own function. It's not always as fast as you think (example coming later). Always prefer native `numpy` functions or vectorised arithmetic.

## Pandas (Sorry this bit won't work without some data)

&nbsp;
<div align="center">
  <img width="400px" src="https://i.pinimg.com/originals/f3/34/ea/f334ea7f813f6178a40368e2706f13dd.gif" />
</div>



In [ ]:
import pandas as pd

In [ ]:
big_data = pd.read_pickle(
    "REDACTED"
)

In [ ]:
big_data

### That example I promised

In [ ]:
import astropy.units as apunits
import astropy.constants as apconst

In [ ]:
def computePeriod(data, keepunit=False, unit=None):
    T = np.sqrt(
        4
        * (data.a.to_numpy() * apconst.R_sun) ** 3
        * np.pi ** 2
        / (apconst.G * (data.Macc.to_numpy() + data.Mcomp.to_numpy()) * apconst.M_sun)
    )
    if unit is not None:
        T = T.to(unit)
    return T if keepunit else T.value

def computePeriod2(data, keepunit=False, unit=None):
    T = np.sqrt(
        4
        * (data.a * apconst.R_sun) ** 3
        * np.pi ** 2
        / (apconst.G * (data.Macc + data.Mcomp * apconst.M_sun))
    )
    if unit is not None:
        T = T.to(unit)
    return T if keepunit else T.value

In [ ]:
%%timeit -r 1 -n 1
periods = computePeriod(big_data, True, apunits.year)

In [ ]:
big_data.apply(computePeriod2, args=(True, apunits.year), axis=1)